In [1]:
import gurobipy as gp
from gurobipy import GRB
import itertools

In [2]:
# Tekijöiden Earthquake, Barrier degradation ja Monolith degradation tilat
outcomes = [
    ['bdbe', 'major'],
    ['fast', 'slow'],
    ['very fast', 'fast', 'slow']
]

# Skenariot
scenarios = list(itertools.product(*outcomes))

# Esimerkkeinä ensimäinen, toinen ja viimeinen
for i in [0, 1, len(scenarios)-1]:
    print(f'Scenario {i+1}: {scenarios[i]}')

Scenario 1: ('bdbe', 'fast', 'very fast')
Scenario 2: ('bdbe', 'fast', 'fast')
Scenario 12: ('major', 'slow', 'slow')


In [3]:
# Uusi malli
model = gp.Model()

# Skenarioille muuttujat
x = model.addVars(scenarios)
model.update()

# Esimerkki
print(f'\n\n{x.items()[0]}')

Academic license - for non-commercial use only - expires 2021-09-29
Using license file /home/anselmi/gurobi.lic


(('bdbe', 'fast', 'very fast'), <gurobi.Var C0>)


In [4]:
### Marginaalit ###

# Alla x.sum summaa ne skenarioit joissa ensimäisen tekijän tila on bdbe, muut voivat olla mitä tahansa (tähti)
model.addConstr(x.sum('bdbe','*','*') >= 0.9912)

# Loput

model.addConstr(x.sum('bdbe','*','*') <= 0.9950)
model.addConstr(x.sum('major','*','*') >= 0.0050)
model.addConstr(x.sum('major','*','*') <= 0.0088)

model.addConstr(x.sum('*','fast','*') >= 0.0787)
model.addConstr(x.sum('*','fast','*') <= 0.2337)
model.addConstr(x.sum('*','slow','*') >= 0.7663)
model.addConstr(x.sum('*','slow','*') <= 0.9213)

model.addConstr(x.sum('*','*','very fast') >= 0.0293)
model.addConstr(x.sum('*','*','very fast') <= 0.2678)
model.addConstr(x.sum('*','*','fast') >= 0.0594)
model.addConstr(x.sum('*','*','fast') <= 0.2695)
model.addConstr(x.sum('*','*','slow') >= 0.4627)
model.addConstr(x.sum('*','*','slow') <= 0.9114)

model.update()

In [5]:
# Muita rajoituksia
model.addConstr(x.sum() == 1)
model.update()

# Tähän rajoituksia lopullisessa versiossa lisää

In [6]:
### Cross Impactit ###

c1 = model.addConstr(x.sum('bdbe','fast','*') >= 0.9544*x.sum('bdbe','*','*')*x.sum('*','fast','*'))
c2 = model.addConstr(x.sum('bdbe','fast','*') <= 0.9963*x.sum('bdbe','*','*')*x.sum('*','fast','*'))

model.addConstr(x.sum('bdbe','slow','*') >= 1.0011*x.sum('bdbe','*','*')*x.sum('*','slow','*'))
model.addConstr(x.sum('bdbe','slow','*') <= 1.0036*x.sum('bdbe','*','*')*x.sum('*','slow','*'))

model.addConstr(x.sum('bdbe','*','very fast') >= 0.9950*x.sum('bdbe','*','*')*x.sum('*','*','very fast'))
model.addConstr(x.sum('bdbe','*','very fast') <= 1.0050*x.sum('bdbe','*','*')*x.sum('*','*','very fast'))

model.addConstr(x.sum('bdbe','*','fast') >= 0.9329*x.sum('bdbe','*','*')*x.sum('*','*','fast'))
model.addConstr(x.sum('bdbe','*','fast') <= 0.9982*x.sum('bdbe','*','*')*x.sum('*','*','fast'))

model.addConstr(x.sum('bdbe','*','slow') >= 0.9975*x.sum('bdbe','*','*')*x.sum('*','*','slow'))
model.addConstr(x.sum('bdbe','*','slow') <= 1.0052*x.sum('bdbe','*','*')*x.sum('*','*','slow'))



model.addConstr(x.sum('major','fast','*') >= 1.5755*x.sum('major','*','*')*x.sum('*','fast','*'))
model.addConstr(x.sum('major','fast','*') <= 6.9785*x.sum('major','*','*')*x.sum('*','fast','*'))

model.addConstr(x.sum('major','slow','*') >= 0.5660*x.sum('major','*','*')*x.sum('*','slow','*'))
model.addConstr(x.sum('major','slow','*') <= 0.8174*x.sum('major','*','*')*x.sum('*','slow','*'))

model.addConstr(x.sum('major','*','very fast') >= 0.9950*x.sum('major','*','*')*x.sum('*','*','very fast'))
model.addConstr(x.sum('major','*','very fast') <= 1.0050*x.sum('major','*','*')*x.sum('*','*','very fast'))

model.addConstr(x.sum('major','*','fast') >= 1.2769*x.sum('major','*','*')*x.sum('*','*','fast'))
model.addConstr(x.sum('major','*','fast') <= 10.1853*x.sum('major','*','*')*x.sum('*','*','fast'))

model.addConstr(x.sum('major','*','slow') >= 0.3406*x.sum('major','*','*')*x.sum('*','*','slow'))
model.addConstr(x.sum('major','*','slow') <= 1.3606*x.sum('major','*','*')*x.sum('*','*','slow'))

model.update()

In [7]:
# Nyt malli oikein määritelty, gurobi varmistaa myös tämän alle sekunnissa
model.setObjective(0, GRB.MAXIMIZE)
model.setParam('NonConvex', 2)
model.optimize()

Changed value of parameter NonConvex to 2
   Prev: -1  Min: -1  Max: 2  Default: -1
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (linux64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 15 rows, 12 columns and 84 nonzeros
Model fingerprint: 0xffa19806
Model has 20 quadratic constraints
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [3e-01, 2e+01]
  QLMatrix range   [1e+00, 1e+00]
  Objective range  [0e+00, 0e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e-03, 1e+00]
Presolve removed 7 rows and 0 columns

Continuous model is non-convex -- solving as a MIP.

Presolve time: 0.00s
Presolved: 335 rows, 90 columns, 1283 nonzeros
Presolved model has 78 bilinear constraint(s)
Variable types: 90 continuous, 0 integer (0 binary)

Root relaxation: objective -0.000000e+00, 49 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth In

In [8]:
# Korvataan c1 ja c2 absurdeilla arvoilla
model.remove(c1)
model.remove(c2)
c1 = model.addConstr(x.sum('bdbe','fast','*') >= 2*x.sum('bdbe','*','*')*x.sum('*','fast','*'))
c2 = model.addConstr(x.sum('bdbe','fast','*') <= 7*x.sum('bdbe','*','*')*x.sum('*','fast','*'))

# Tälläkin kertaa malli antaa nopeasti vastauksen
model.optimize()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (linux64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 15 rows, 12 columns and 84 nonzeros
Model fingerprint: 0x13907882
Model has 20 quadratic constraints
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [3e-01, 2e+01]
  QLMatrix range   [1e+00, 1e+00]
  Objective range  [0e+00, 0e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e-03, 1e+00]
Presolve removed 7 rows and 0 columns

Continuous model is non-convex -- solving as a MIP.


MIP start from previous solve did not produce a new incumbent solution

Presolve time: 0.00s
Presolved: 335 rows, 90 columns, 1283 nonzeros
Presolved model has 78 bilinear constraint(s)
Variable types: 90 continuous, 0 integer (0 binary)

Root relaxation: infeasible, 44 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    Best

In [9]:
# Tämä ei kuitenkaan aina päde. Palautetaan alkuperäinen c1 ja tiukennetaan c2
model.remove(c1)
model.remove(c2)
c1 = model.addConstr(x.sum('bdbe','fast','*') >= 0.9544*x.sum('bdbe','*','*')*x.sum('*','fast','*'))
c2 = model.addConstr(x.sum('bdbe','fast','*') <= 0.9550*x.sum('bdbe','*','*')*x.sum('*','fast','*'))
model.update()

# Nyt miettimiseen meneekin hieman aikaa
model.optimize()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (linux64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 15 rows, 12 columns and 84 nonzeros
Model fingerprint: 0xcb7c007b
Model has 20 quadratic constraints
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [3e-01, 2e+01]
  QLMatrix range   [1e+00, 1e+00]
  Objective range  [0e+00, 0e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e-03, 1e+00]
Presolve removed 7 rows and 0 columns

Continuous model is non-convex -- solving as a MIP.


MIP start from previous solve did not produce a new incumbent solution
Processed MIP start in 1.04 seconds

Presolve time: 0.00s
Presolved: 335 rows, 90 columns, 1283 nonzeros
Presolved model has 78 bilinear constraint(s)
Variable types: 90 continuous, 0 integer (0 binary)

Root relaxation: objective -0.000000e+00, 74 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Exp

 1193839  1126   -0.00000   49   52          -   -0.00000      -  26.9  455s
 1206800  1249   -0.00000   35   34          -   -0.00000      -  26.9  460s
 1219800  1247 infeasible   36               -   -0.00000      -  26.9  465s
 1232768  1121 infeasible   48               -   -0.00000      -  26.9  470s
 1246047  1019   -0.00000   51   42          -   -0.00000      -  26.9  475s
 1259190  1008   -0.00000   41   34          -   -0.00000      -  26.9  480s
 1272111  1166 infeasible   58               -   -0.00000      -  26.9  485s
 1285210  1213 infeasible   51               -   -0.00000      -  26.9  490s
 1298339  1177   -0.00000   39   35          -   -0.00000      -  26.9  495s
 1310369   974   -0.00000   56   59          -   -0.00000      -  27.0  500s
 1324898  1056   -0.00000   42   38          -   -0.00000      -  27.0  505s
 1337974   935 infeasible   47               -   -0.00000      -  27.0  510s
 1352134   960 infeasible   50               -   -0.00000      -  27.0  515s

 2607311  1822   -0.00000   87   67          -   -0.00000      -  27.2  990s
 2620509  1831   -0.00000   75   55          -   -0.00000      -  27.2  995s
 2632675  1624 infeasible   78               -   -0.00000      -  27.2 1000s
 2645314  1360   -0.00000   67   28          -   -0.00000      -  27.2 1005s
 2657349  1467 infeasible   79               -   -0.00000      -  27.2 1010s
 2670291  1279   -0.00000   41   36          -   -0.00000      -  27.3 1015s
 2683578  1262 infeasible   80               -   -0.00000      -  27.3 1020s
 2695294  1074   -0.00000   49   63          -   -0.00000      -  27.3 1025s
 2707486  1032 infeasible   72               -   -0.00000      -  27.3 1030s
 2720595   968 infeasible   67               -   -0.00000      -  27.3 1035s
 2732358   848   -0.00000   47   72          -   -0.00000      -  27.3 1040s
 2745578   797   -0.00000   60   55          -   -0.00000      -  27.3 1045s
 2757655   771   -0.00000   56   66          -   -0.00000      -  27.3 1050s

 4044403  1028   -0.00000   42   49          -   -0.00000      -  27.1 1525s
 4058743  1060 infeasible   53               -   -0.00000      -  27.1 1530s
 4071670  1025 infeasible   54               -   -0.00000      -  27.1 1535s
 4086092   873 infeasible   49               -   -0.00000      -  27.1 1540s
 4099020  1209 infeasible   51               -   -0.00000      -  27.1 1545s
 4113533  1516   -0.00000   51   57          -   -0.00000      -  27.1 1550s
 4126323  1535 infeasible   52               -   -0.00000      -  27.1 1555s
 4139412  1746 infeasible   58               -   -0.00000      -  27.1 1560s
 4154164  1563 infeasible   75               -   -0.00000      -  27.1 1565s
 4168005  1674 infeasible   52               -   -0.00000      -  27.1 1570s
 4180797  1675 infeasible   69               -   -0.00000      -  27.1 1575s
 4195413  1506 infeasible   76               -   -0.00000      -  27.1 1580s
 4208548  1501 infeasible   54               -   -0.00000      -  27.1 1585s

 5514826  1574   -0.00000   60   66          -   -0.00000      -  26.9 2060s
 5527499  1693   -0.00000   61   36          -   -0.00000      -  26.9 2065s
 5541656  1925 infeasible   64               -   -0.00000      -  26.9 2070s
 5554789  2032 infeasible   59               -   -0.00000      -  26.9 2075s
 5567904  2014 infeasible   58               -   -0.00000      -  26.9 2080s
 5581324  1918   -0.00000   57   36          -   -0.00000      -  26.9 2085s
 5595573  1842   -0.00000   63   35          -   -0.00000      -  26.9 2090s
 5608369  1754   -0.00000   65   53          -   -0.00000      -  26.9 2095s
 5621612  1875 infeasible   73               -   -0.00000      -  26.9 2100s
 5635010  1950   -0.00000   72   44          -   -0.00000      -  26.9 2105s
 5648182  2125 infeasible   66               -   -0.00000      -  26.9 2110s
 5662310  2102 infeasible   74               -   -0.00000      -  26.9 2115s
 5675551  1989   -0.00000   82   53          -   -0.00000      -  26.9 2120s

...

 14224957  1694 infeasible   74               -   -0.00000      -  26.8 5255s
 14236911  1900   -0.00000   68   44          -   -0.00000      -  26.8 5260s
 14251490  2122   -0.00000   63   55          -   -0.00000      -  26.8 5265s
 14265292  2470 infeasible   79               -   -0.00000      -  26.8 5270s
 14280642  2560 infeasible   67               -   -0.00000      -  26.8 5275s
 14294804  2427   -0.00000   83   28          -   -0.00000      -  26.8 5280s
 14309389  2279   -0.00000   84   64          -   -0.00000      -  26.8 5285s
 14323481  2174   -0.00000   70   14          -   -0.00000      -  26.8 5290s
 14337891  2059 infeasible   87               -   -0.00000      -  26.8 5295s
 14353470  1952 infeasible   65               -   -0.00000      -  26.8 5300s
 14366060  1869   -0.00000   62   55          -   -0.00000      -  26.8 5305s
 14380443  1817   -0.00000   63   28          -   -0.00000      -  26.8 5310s
 14394663  1734 infeasible   84               -   -0.00000      

 15652011  1391 infeasible   57               -   -0.00000      -  26.8 5785s
 15665164  1422 infeasible   66               -   -0.00000      -  26.8 5790s
 15678201  1426 infeasible   62               -   -0.00000      -  26.8 5795s
 15691355  1315   -0.00000   70   48          -   -0.00000      -  26.8 5800s
 15703087  1266 infeasible   61               -   -0.00000      -  26.8 5805s
 15714936  1115   -0.00000   59   54          -   -0.00000      -  26.8 5810s
 15728337   961 infeasible   71               -   -0.00000      -  26.8 5815s
 15741536  1044 infeasible   56               -   -0.00000      -  26.8 5820s
 15754518   939 infeasible   53               -   -0.00000      -  26.8 5825s
 15768952   936 infeasible   78               -   -0.00000      -  26.8 5830s
 15782142   996 infeasible   58               -   -0.00000      -  26.8 5835s
 15796342  1199   -0.00000   56   27          -   -0.00000      -  26.8 5840s
 15809473  1212 infeasible   67               -   -0.00000      

 17099280   877 infeasible   56               -   -0.00000      -  26.8 6315s
 17112544   829 infeasible   44               -   -0.00000      -  26.8 6320s
 17126579   819   -0.00000   53   57          -   -0.00000      -  26.8 6325s
 17139812  1005   -0.00000   56   45          -   -0.00000      -  26.8 6330s
 17152805   883   -0.00000   43   35          -   -0.00000      -  26.8 6335s
 17164622   760 infeasible   60               -   -0.00000      -  26.8 6340s
 17176506   670   -0.00000   34   43          -   -0.00000      -  26.8 6345s
 17189873   836   -0.00000   39   27          -   -0.00000      -  26.8 6350s
 17204417   960   -0.00000   50   44          -   -0.00000      -  26.8 6355s
 17217643   974 infeasible   53               -   -0.00000      -  26.8 6360s
 17230853   900 infeasible   57               -   -0.00000      -  26.8 6365s
 17243915   749 infeasible   52               -   -0.00000      -  26.8 6370s
 17256921   591 infeasible   60               -   -0.00000      

 18530557  1444   -0.00000   59   37          -   -0.00000      -  26.8 6845s
 18543614  1294 infeasible   52               -   -0.00000      -  26.8 6850s
 18557241  1219 infeasible   45               -   -0.00000      -  26.8 6855s
 18569930  1419   -0.00000   50   54          -   -0.00000      -  26.8 6860s
 18582896  1506   -0.00000   49   51          -   -0.00000      -  26.8 6865s
 18595856  1418 infeasible   59               -   -0.00000      -  26.8 6870s
 18608827  1260 infeasible   63               -   -0.00000      -  26.8 6875s
 18619450  1167   -0.00000   53   63          -   -0.00000      -  26.8 6880s
 18631793  1145 infeasible   47               -   -0.00000      -  26.8 6885s
 18646479  1387 infeasible   55               -   -0.00000      -  26.8 6890s
 18659393  1393   -0.00000   57   27          -   -0.00000      -  26.8 6895s
 18672702  1386   -0.00000   65   58          -   -0.00000      -  26.8 6900s
 18687690  1605   -0.00000   50   27          -   -0.00000      

 19940614   835 infeasible   61               -   -0.00000      -  26.8 7375s
 19953629   840 infeasible   76               -   -0.00000      -  26.8 7380s
 19966737   786   -0.00000   52   59          -   -0.00000      -  26.8 7385s
 19981440   914 infeasible   63               -   -0.00000      -  26.8 7390s
 19995637   841 infeasible   53               -   -0.00000      -  26.8 7395s
 20008723   680 infeasible   48               -   -0.00000      -  26.8 7400s
 20021625   617 infeasible   48               -   -0.00000      -  26.8 7405s
 20035023   572 infeasible   45               -   -0.00000      -  26.8 7410s
 20047884   364   -0.00000   44   48          -   -0.00000      -  26.8 7415s
 20061938   501   -0.00000   40   34          -   -0.00000      -  26.8 7420s
 20073681   652 infeasible   54               -   -0.00000      -  26.8 7425s
 20086815   574 infeasible   55               -   -0.00000      -  26.8 7430s
 20099687   597 infeasible   63               -   -0.00000      